In [1]:
import pandas as pd
import os
import glob

# ファイル探索（../src/ フォルダ内）
file_pattern = os.path.join('..', 'src', 'iMaPP_database*.xlsx')
matched_files = glob.glob(file_pattern)

if matched_files:
    file_path = matched_files[0]
    df_mapp_t = pd.read_excel(file_path, sheet_name='MaPP_T')
    print(f"✅ 読込成功: {file_path}")
else:
    raise FileNotFoundError("❌ 該当ファイルが見つかりませんでした。")


✅ 読込成功: ..\src\iMaPP_database-2024-12-2.xlsx


In [2]:
# 欠損値をすべて0で埋める
df_mapp_t = df_mapp_t.fillna(0)


In [11]:
# サブカテゴリに該当しない主要ツールのみ抽出（_Tで終わるが、_HH_Tなどではなく、かつ SUM_17_T でもない）
main_tool_cols = [
    col for col in df_mapp_t.columns 
    if col.endswith('_T') 
    and not any(suffix in col for suffix in ['_HH_T', '_Corp_T', '_Gen_T', '_FX_T', '_FCD_T']) 
    and col != 'SUM_17_T'
]

# ロング形式に変換
df_main_long = df_mapp_t.melt(
    id_vars=['iso2', 'Year', 'Month'],
    value_vars=main_tool_cols,
    var_name='MaPP_Tool_Full',
    value_name='OBS_Tightening'
)

# ツール名のみ抽出（_Tを除去）
df_main_long['MaPP_Tool'] = df_main_long['MaPP_Tool_Full'].str.replace('_T', '', regex=False)
df_main_long['Subcategory'] = None

# 最終整形
df_main_tools = df_main_long[['iso2', 'Year', 'Month', 'MaPP_Tool', 'Subcategory', 'OBS_Tightening']]
df_main_tools.head()


,iso2,Year,Month,MaPP_Tool,Subcategory,OBS_Tightening
0,AL,1990,1,CCB,None,0.0
1,AL,1990,2,CCB,None,0.0
2,AL,1990,3,CCB,None,0.0
3,AL,1990,4,CCB,None,0.0
4,AL,1990,5,CCB,None,0.0


In [12]:
# サブカテゴリ付きのカラム抽出
subcat_tool_cols = [
    col for col in df_mapp_t.columns 
    if any(suffix in col for suffix in ['_HH_T', '_Corp_T', '_Gen_T', '_FX_T', '_FCD_T'])
]

# ロング形式に変換
df_subcat_long = df_mapp_t.melt(
    id_vars=['iso2', 'Year', 'Month'],
    value_vars=subcat_tool_cols,
    var_name='MaPP_Tool_Full',
    value_name='OBS_Tightening'
)

# MaPP_Tool, Subcategory に分解（例：Capital_HH_T → Capital / HH）
df_subcat_long[['MaPP_Tool', 'Subcategory']] = df_subcat_long['MaPP_Tool_Full'].str.extract(r'(\w+?)_(HH|Corp|Gen|FX|FCD)_T')

# 整形
df_subcat_tools = df_subcat_long[['iso2', 'Year', 'Month', 'MaPP_Tool', 'Subcategory', 'OBS_Tightening']]
df_subcat_tools.head()


,iso2,Year,Month,MaPP_Tool,Subcategory,OBS_Tightening
0,AL,1990,1,Capital,Gen,0.0
1,AL,1990,2,Capital,Gen,0.0
2,AL,1990,3,Capital,Gen,0.0
3,AL,1990,4,Capital,Gen,0.0
4,AL,1990,5,Capital,Gen,0.0


In [13]:
# 月を四半期に変換
df_main_tools['Quarter'] = df_main_tools['Month'].apply(lambda m: f"Q{((int(m)-1)//3)+1}")

# 四半期キー列を作成
df_main_tools['Year_Quarter'] = df_main_tools['Year'].astype(str) + df_main_tools['Quarter']


In [14]:
# 四半期内の合計値（何回引き締めがあったか）
df_quarterly_agg = (
    df_main_tools
    .groupby(['iso2', 'MaPP_Tool', 'Year_Quarter'], as_index=False)
    .agg({'OBS_Tightening': 'sum'})
)

# 列名変更（明示）
df_quarterly_agg = df_quarterly_agg.rename(columns={'OBS_Tightening': 'OBS_Tightening_Aggregated'})


In [15]:
# 四半期内に1回でも引き締めがあったら 1
df_quarterly_bin = (
    df_main_tools
    .groupby(['iso2', 'MaPP_Tool', 'Year_Quarter'], as_index=False)
    .agg({'OBS_Tightening': 'max'})
)

df_quarterly_bin = df_quarterly_bin.rename(columns={'OBS_Tightening': 'OBS_Tightening_Binary'})


In [16]:
# CSVとして保存
df_quarterly_agg.to_csv('iMaPP_main_tools_quarterly_agg.csv', index=False, encoding='utf-8-sig')
df_quarterly_bin.to_csv('iMaPP_main_tools_quarterly_bin.csv', index=False, encoding='utf-8-sig')

print("✅ 四半期データ（合計 & バイナリ）をそれぞれ保存しました。")


✅ 四半期データ（合計 & バイナリ）をそれぞれ保存しました。
